In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

"""
Load Customer data into Aerospike

"""

# Import libraries

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import aerospike
import json

print(__doc__)

def load_data():
            
    # Read input dataset using pandas
    data = pd.read_excel("/home/azureuser/aerospike/data/customer/customer_address.xls")
    
    # Drop serial number from csv file    
    #data = data.drop([data.columns[0]], axis=1)
    
    # Select numeric data alone from data object
    numeric_data = data.loc[:, data.dtypes != object]
    
    # Select string data alone from data object to trim
    trim_data = data.select_dtypes(['object'])
    
    # Trim all string objects
    trim_data = trim_data.apply(lambda x: x.str.strip())
    
    #Merge numeric and string data in to data object
    data = pd.concat([numeric_data, trim_data], axis=1)
    
    #Split dataframe into key and records
    key = data['cust_id']
    
    record_json = data.to_dict(orient='records')
    
    config = {
      'hosts': [ ('127.0.0.1', 3000) ]
    }

    # Create a client and connect it to the cluster
    try:
        client = aerospike.client(config).connect()
    except:
        import sys
        print("failed to connect to the cluster with", config['hosts'])
        sys.exit(1)
    
    # Records are addressable via a tuple of (namespace, set, key)
    for pk, bin in zip(key, record_json):
        string = "('dev', 's1ad', "
        string +=`pk`
        string = string + ")"
        
        from ast import literal_eval as make_tuple
        string = make_tuple(string)
        
        try:
            client = aerospike.client(config).connect()
            # Write a record
            client.put(string, bin)
        except Exception as e:
            import sys
            print("error: {0}".format(e), file=sys.stderr)

        # Read a record after insert for verification
        (key, metadata, record) = client.get(string)
        print (key, record)
        
        # Close the connection to the Aerospike cluster
        client.close()

if __name__ == '__main__':
    #load_data()
    data = pd.read_excel("/home/azureuser/aerospike/data/customer/customer_address.xls")
    print (data)


Load Customer data into Aerospike




XLRDError: File size is 0 bytes